In [1]:
%pip install numpy pandas scikit-learn

  Using cached pandas-2.3.1-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (91 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 85.8 MB/s eta 0:00:00
Using cached pandas-2.3.1-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.1 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 83.6 MB/s eta 0:00:00
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.2/35.2 MB 91.1 MB/s eta 0:00:00ta 0:00:01
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [scikit-learn] [scikit-learn]

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade

# Task 1

In [2]:
import numpy as np
import pandas as pd

In [3]:
# --- 1. Define Logistic Regression Functions ---


def sigmoid(z):
    """
    Compute the sigmoid of z.
    Clips z to prevent overflow in exp.
    """
    # Clip z to prevent overflow
    z = np.clip(z, -500, 500)
    return 1 / (1 + np.exp(-z))


def loss(y, y_hat):
    """
    Compute the Log Loss (Binary Cross-Entropy).
    """
    # Small epsilon to avoid log(0)
    epsilon = 1e-15
    y_hat = np.clip(y_hat, epsilon, 1 - epsilon)  # Clip predictions
    m = y.shape[0]
    cost = -np.sum(y * np.log(y_hat) + (1 - y) * np.log(1 - y_hat)) / m
    return cost


def gradients(X, y, y_hat):
    """
    Compute the gradients of the loss with respect to weights and bias.
    """
    m = X.shape[0]  # Number of examples
    dw = (1 / m) * np.dot(X.T, (y_hat - y))  # Gradient w.r.t weights
    db = (1 / m) * np.sum(y_hat - y)  # Gradient w.r.t bias
    return dw, db


def train(X, y, bs, epochs, lr):
    """
    Train the Logistic Regression model.
    """
    # Initialize weights and bias
    n_features = X.shape[1]
    w = np.zeros((n_features, 1))  # Weights
    b = 0  # Bias

    # Reshape y to be a column vector
    y = y.reshape(X.shape[0], 1)

    # Training loop
    costs = []
    for epoch in range(epochs):
        # Forward pass: compute predicted y
        z = np.dot(X, w) + b
        y_hat = sigmoid(z)

        # Compute cost
        cost = loss(y, y_hat)
        costs.append(cost)

        # Backward pass: compute gradients
        dw, db = gradients(X, y, y_hat)

        # Update parameters
        w -= lr * dw
        b -= lr * db

        # Print cost every 100 epochs
        if epoch % 100 == 0:
            print(f"Epoch {epoch}, Cost: {cost}")

    # Return final parameters
    return w, b


def predict(X, w, b):
    """
    Predict labels for dataset X using learned parameters w and b.
    """
    z = np.dot(X, w) + b
    y_hat = sigmoid(z)
    # Convert probabilities to class predictions (0 or 1)
    preds = (y_hat > 0.5).astype(int)
    return preds.flatten()  # Return as 1D array


In [ ]:
# --- 2. Load Data ---

print("Loading data...")
# Load training data
train_df = pd.read_csv("data/train_tfidf_features.csv")
X_train = train_df.drop(["id", "label"], axis=1).values  # Features (columns 0-4999)
y_train = train_df["label"].values  # Labels

# Load test data
test_df = pd.read_csv("data/test_tfidf_features.csv")
X_test = test_df.drop(["id"], axis=1).values  # Features (columns 0-4999)
test_ids = test_df["id"].values  # Test IDs for submission

print(f"Training set shape: {X_train.shape}, Labels shape: {y_train.shape}")
print(f"Test set shape: {X_test.shape}")

# --- 3. Train the Model ---
print("\nTraining Logistic Regression model...")
# Hyperparameters
learning_rate = 0.01
num_epochs = 1000  # Increased epochs for better convergence
# Note: Batch size (bs) is not used in this simple implementation,
# but it's a parameter for the train function as requested.
# We'll use the full batch (gradient descent).
w_opt, b_opt = train(X_train, y_train, bs=None, epochs=num_epochs, lr=learning_rate)

# --- 4. Make Predictions on Test Set ---
print("\nMaking predictions on test set...")
test_predictions = predict(X_test, w_opt, b_opt)

# --- 5. Create Submission File ---
print("\nCreating submission file: LogRed_Prediction.csv")
submission_df = pd.DataFrame({"row ID": test_ids, "label": test_predictions})
# Save to CSV without index
submission_df.to_csv("LogRed_Prediction.csv", index=False)
print("Submission file 'LogRed_Prediction.csv' created successfully!")


Loading data...
Training set shape: (17184, 5000), Labels shape: (17184,)
Test set shape: (4296, 5000)

Training Logistic Regression model...
Epoch 0, Cost: 0.6931471805599453
Epoch 100, Cost: 0.6818685756190892
Epoch 200, Cost: 0.6750098654665818
Epoch 300, Cost: 0.6708079172240665
Epoch 400, Cost: 0.668206170014174
Epoch 500, Cost: 0.6665716538876123
Epoch 600, Cost: 0.6655240744963636
Epoch 700, Cost: 0.6648340128593146
Epoch 800, Cost: 0.6643624730359983
Epoch 900, Cost: 0.6640249036359575

Making predictions on test set...

Creating submission file: LogRed_Prediction.csv
Submission file 'LogRed_Prediction.csv' created successfully!


# Task 2

In [6]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report

In [ ]:
# 2. Load CSVs
train_feats = pd.read_csv(
    "data/train_tfidf_features.csv"
)  # columns: id, label, 0,1,...,4999
test_feats = pd.read_csv("data/test_tfidf_features.csv")  # columns: id, 0,1,...,4999
train_raw = pd.read_csv("data/train.csv")
test_raw = pd.read_csv("data/test.csv")

# 3. Extract feature arrays and labels
X_full = train_feats.drop(
    columns=["id", "label"], errors="ignore"
).values  # shape (n_train,5000)
y_full = train_feats["label"].values  # shape (n_train,)
X_test = test_feats.drop(columns=["id"], errors="ignore").values  # shape (n_test,5000)
test_ids = test_feats["id"].values  # shape (n_test,)

# 4. Train/validation split
X_train, X_val, y_train, y_val = train_test_split(
    X_full, y_full, test_size=0.2, random_state=42, stratify=y_full
)
print(f"Shapes → Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}\n")

# 5. PCA + KNN for each component count
for comps in [2000, 1000, 500, 100]:
    n_comp = min(comps, X_train.shape[1])
    pca = PCA(n_components=n_comp, random_state=42)
    X_tr_pca = pca.fit_transform(X_train)
    X_val_pca = pca.transform(X_val)
    X_te_pca = pca.transform(X_test)

    assert X_tr_pca.shape[1] == n_comp
    assert X_val_pca.shape[1] == n_comp
    assert X_te_pca.shape[1] == n_comp

    knn = KNeighborsClassifier(n_neighbors=2)
    knn.fit(X_tr_pca, y_train)

    # Validation
    yv = knn.predict(X_val_pca)
    acc = accuracy_score(y_val, yv)
    m_f1 = f1_score(y_val, yv, average="macro")
    print(f"[Task 2] PCA {n_comp} → Val Accuracy: {acc:.4f}, Macro F1: {m_f1:.4f}")
    print(classification_report(y_val, yv))

    # Test predictions & save
    y_test_pred = knn.predict(X_te_pca)
    submission_df = pd.DataFrame({"row ID": test_ids, "label": y_test_pred})
    filename = f"PCA_{n_comp}_KNN.csv"
    submission_df.to_csv(filename, index=False)
    print(f"Saved {filename}")
    print("Preview:")
    print(submission_df.head(10))
    print("-" * 60 + "\n")


Shapes → Train: (13747, 5000), Val: (3437, 5000), Test: (4296, 5000)

[Task 2] PCA 2000 → Val Accuracy: 0.4987, Macro F1: 0.4979
              precision    recall  f1-score   support

           0       0.67      0.37      0.48      2127
           1       0.41      0.71      0.52      1310

    accuracy                           0.50      3437
   macro avg       0.54      0.54      0.50      3437
weighted avg       0.57      0.50      0.49      3437

Saved PCA_2000_KNN.csv
Preview:
      id  label
0  17185      1
1  17186      1
2  17187      1
3  17188      1
4  17189      0
5  17190      1
6  17191      1
7  17192      0
8  17193      0
9  17194      1
------------------------------------------------------------

[Task 2] PCA 1000 → Val Accuracy: 0.5982, Macro F1: 0.5568
              precision    recall  f1-score   support

           0       0.66      0.73      0.69      2127
           1       0.47      0.38      0.42      1310

    accuracy                           0.60      34

# Task 3

In [ ]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import train_test_split
import time

In [ ]:
def clean_text(text):
    """Light text cleaning"""
    if pd.isna(text):
        return ""
    text = str(text).lower()
    # Remove URLs
    text = re.sub(r"http\S+|www\S+", "", text)
    # Remove extra whitespace
    text = re.sub(r"\s+", " ", text).strip()
    return text


def optimized_tfidf_lr_approach():
    print("Loading data...")
    train_df = pd.read_csv("data/train.csv")
    test_df = pd.read_csv("data/test.csv")

    # Identify text column
    text_col = [c for c in train_df.columns if c not in ("id", "label")][0]

    # Prepare data
    X_train_raw = train_df[text_col].apply(clean_text)
    y_train = train_df["label"].values
    X_test_raw = test_df[text_col].apply(clean_text)
    test_ids = test_df["id"].values

    # Create validation set
    X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
        X_train_raw, y_train, test_size=0.2, random_state=42, stratify=y_train
    )

    print("Building pipeline...")
    # Optimized pipeline
    pipeline = Pipeline(
        [
            (
                "tfidf",
                TfidfVectorizer(
                    max_features=15000,  # More features for better coverage
                    ngram_range=(1, 3),  # Unigrams, bigrams, and trigrams
                    min_df=2,  # Ignore very rare terms
                    max_df=0.95,  # Ignore very common terms
                    stop_words="english",  # Remove English stop words
                    sublinear_tf=True,  # Apply sublinear TF scaling
                    strip_accents="unicode",  # Handle unicode characters
                ),
            ),
            (
                "feature_selection",
                SelectKBest(score_func=chi2, k=5000),
            ),  # Select top 5000 features
            (
                "classifier",
                LogisticRegression(
                    C=1.0,  # Regularization parameter
                    random_state=42,
                    max_iter=1000,
                    class_weight="balanced",  # Handle class imbalance
                ),
            ),
        ]
    )

    print("Training model...")
    start_time = time.time()

    # Train the pipeline
    pipeline.fit(X_train_split, y_train_split)

    training_time = time.time() - start_time
    print(f"Training completed in {training_time:.2f} seconds")

    # Validate
    print("Validating model...")
    y_val_pred = pipeline.predict(X_val_split)
    val_f1 = f1_score(y_val_split, y_val_pred, average="macro")

    print(f"Validation Macro F1 Score: {val_f1:.4f}")
    print("\nDetailed Validation Report:")
    print(classification_report(y_val_split, y_val_pred))

    # Feature analysis
    print("\nAnalyzing important features...")
    # Get feature names and coefficients
    feature_names = pipeline.named_steps["tfidf"].get_feature_names_out()
    selected_features = pipeline.named_steps["feature_selection"].get_support()
    selected_feature_names = feature_names[selected_features]
    coefficients = pipeline.named_steps["classifier"].coef_[0]

    # Top features for each class
    feature_coef_pairs = list(zip(selected_feature_names, coefficients))
    feature_coef_pairs.sort(key=lambda x: x[1])

    print("Top 10 features for non-hateful class:")
    for feature, coef in feature_coef_pairs[:10]:
        print(f"  {feature}: {coef:.4f}")

    print("\nTop 10 features for hateful class:")
    for feature, coef in feature_coef_pairs[-10:]:
        print(f"  {feature}: {coef:.4f}")

    # Retrain on full dataset
    print("\nRetraining on full dataset...")
    pipeline.fit(X_train_raw, y_train)

    # Predict on test set
    print("Making predictions on test set...")
    y_test_pred = pipeline.predict(X_test_raw)

    # Save predictions
    submission_df = pd.DataFrame({"row ID": test_ids, "label": y_test_pred})
    submission_df.to_csv("Optimized_TFIDF_LR_Prediction.csv", index=False)
    print("Predictions saved to Optimized_TFIDF_LR_Prediction.csv")

    # Summary
    print("\n" + "=" * 60)
    print("OPTIMIZED TF-IDF + LOGISTIC REGRESSION SUMMARY")
    print("=" * 60)
    print(f"Validation Macro F1 Score: {val_f1:.4f}")
    print(f"Training Time: {training_time:.2f} seconds")
    print(f"Total Features Generated: 15000")
    print(f"Features Selected: 5000")
    print(f"N-gram Range: (1,3)")
    print("Model: Logistic Regression with L2 regularization")
    print("File Saved: Optimized_TFIDF_LR_Prediction.csv")

    return val_f1, pipeline


# Run the approach
if __name__ == "__main__":
    print("Starting Optimized TF-IDF + Logistic Regression Approach")
    print("Expected training time: 10-20 minutes")
    print("=" * 60)

    val_score, model = optimized_tfidf_lr_approach()

    print(f"\nFinal Validation Score: {val_score:.4f}")
    print("This approach typically achieves 0.75-0.82 Macro F1 on hate speech datasets")


Starting Optimized TF-IDF + Logistic Regression Approach
Expected training time: 10-20 minutes
Loading data...
Building pipeline...
Training model...
Training completed in 0.59 seconds
Validating model...
Validation Macro F1 Score: 0.6998

Detailed Validation Report:
              precision    recall  f1-score   support

           0       0.78      0.74      0.76      2127
           1       0.61      0.67      0.64      1310

    accuracy                           0.71      3437
   macro avg       0.70      0.70      0.70      3437
weighted avg       0.72      0.71      0.71      3437


Analyzing important features...
Top 10 features for non-hateful class:
  cruz: -2.8899
  people: -2.7733
  abortion: -2.4280
  sure: -2.3018
  optics: -2.2371
  bjp: -2.2140
  india: -2.0686
  trump: -1.9451
  prolife: -1.9401
  white house: -1.8961

Top 10 features for hateful class:
  muslims: 2.9090
  blacks: 2.9797
  islam: 2.9909
  faggots: 3.0098
  deport: 3.0350
  illegal: 3.0943
  whitegenocid